In [1]:
import time
import pandas as pd
import json
from kafka import KafkaConsumer
# TOPIC_NAME = 'user_behavior'
TOPIC_NAME = 'buy_cnt_per_hour'
# Create a KafkaConsumer with deserializer
consumer = KafkaConsumer(
    bootstrap_servers=["10.237.96.122:9092"],
    # auto_offset_reset='latest', 
    auto_offset_reset='earliest', 
    # receive_buffer_bytes=160*1024,
    enable_auto_commit=False,
    value_deserializer=lambda m: json.loads(m.decode('ascii')),
)

# Subscribe to a specific topic
consumer.subscribe(topics=TOPIC_NAME)
N_sample = 10
lst_record = []
while True:
    msg_poll = consumer.poll(timeout_ms=0)
    for k, v in msg_poll.items():
        for msg in v: 
            topic = msg.topic
            partition = msg.partition
            offset = msg.offset
            value_msg = msg.value
            try:
                lst_record.append(value_msg)
            except Exception as e:
                print(f"Error: {e}")
                continue
    if len(lst_record) >= N_sample:
        break

df_record = pd.DataFrame(lst_record)
df_record.describe()

In [3]:
df_record.head()

,role_id,behavior,ts
0,47,click,2024-11-24T23:33:11.698844
1,38,like,2024-11-24T23:33:11.700012
2,48,share,2024-11-24T23:33:11.700978
3,72,share,2024-11-24T23:33:11.701693
4,97,share,2024-11-24T23:33:11.701831
